Required initial steps:
    
    * The text file from https://afltables.com/afl/stats/biglists/bg3.txt is saved as llist.txt
    * The file is then processed through the bash script file a_updater.sh to convert it to a csv and make other modifications.
    * The bash file saves the processed file as newlist.csv, ready to be imported from within Python

In [2]:
import pandas as pd
cols = ['seq','date','round','team1','gls_1','bhs_1','pts_1','team2','gls_2','bhs_2','pts_2','venue']

In [3]:
upd_round = pd.read_csv("newlist.csv", names = cols)
trans = upd_round

In [4]:
# Modify index to start at 1 
trans.index += 1
trans.tail()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue
15313,15313,22-Jun-2018,R14,Port Adelaide,11,9,75,Melbourne,9,11,65,Adelaide Oval
15314,15314,23-Jun-2018,R14,Hawthorn,13,18,96,Gold Coast,5,13,43,York Park
15315,15315,23-Jun-2018,R14,Brisbane Lions,12,10,82,GW Sydney,16,13,109,Gabba
15316,15316,23-Jun-2018,R14,Western Bulldogs,11,9,75,North Melbourne,12,5,77,Docklands
15317,15317,24-Jun-2018,R14,Collingwood,11,13,79,Carlton,9,5,59,M.C.G.


In [5]:
# Add states
nsw = ['S.C.G.','Sydney Showground','Stadium Australia','Blacktown','Albury']
qld = ['Gabba','Carrara','Cazaly\'s Stadium','Brisbane Exhibition']
sa = ['Football Park','Adelaide Oval']
wa = ['W.A.C.A.','Perth Stadium','Subiaco']
nt = ['Marrara Oval','Traeger Park']
tas = ['York Park','Bellerive Oval','North Hobart']
act = ['Manuka Oval','Bruce Stadium']
intl = ['Wellington','Jiangwan Stadium']

In [6]:
def locations(x):
    if x in nsw:
        return "NSW"
    elif x in sa:
        return "SA"
    elif x in wa:
        return "WA"
    elif x in nt:
        return "NT"
    elif x in tas:
        return "TAS"
    elif x in intl:
        return "INTL"
    elif x in act:
        return "ACT"
    elif x in qld:
        return "QLD"
    else: 
        return "VIC"

In [7]:
trans["state"] = trans["venue"].apply(locations)
trans["state"].value_counts()

VIC     12990
WA        631
SA        564
NSW       503
QLD       465
TAS        89
ACT        47
NT         23
INTL        5
Name: state, dtype: int64

In [8]:
trans.tail()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue,state
15313,15313,22-Jun-2018,R14,Port Adelaide,11,9,75,Melbourne,9,11,65,Adelaide Oval,SA
15314,15314,23-Jun-2018,R14,Hawthorn,13,18,96,Gold Coast,5,13,43,York Park,TAS
15315,15315,23-Jun-2018,R14,Brisbane Lions,12,10,82,GW Sydney,16,13,109,Gabba,QLD
15316,15316,23-Jun-2018,R14,Western Bulldogs,11,9,75,North Melbourne,12,5,77,Docklands,VIC
15317,15317,24-Jun-2018,R14,Collingwood,11,13,79,Carlton,9,5,59,M.C.G.,VIC


## 1. HVAR & AGGR

In [9]:
trans['hvar'] = trans['pts_1'] - trans['pts_2']
trans['aggr'] = trans['pts_1'] + trans['pts_2'] 

## 2. YEAR & MONTH

In [10]:
import time
import datetime

In [11]:
trans['date'] = pd.to_datetime(trans['date'])
trans['year'], trans['month'] = trans['date'].dt.year, trans['date'].dt.month

## 3. ABBREVIATED CLUB NAMES

In [12]:
adelaide = ['Adelaide']
brisbane = ['Brisbane Bears','Brisbane Lions']
carlton = ['Carlton']
collingwood = ['Collingwood']
essendon = ['Essendon']
fitzroy = ['Fitzroy']
footscray = ['Footscray']
fremantle = ['Fremantle']
geelong = ['Geelong']
gold_coast = ['Gold Coast']
gws = ['GW Sydney']
hawthorn = ['Hawthorn']
melbourne = ['Melbourne']
north_melbourne = ['North Melbourne','Kangaroos']
port_adelaide = ['Port Adelaide']
richmond = ['Richmond']
south_melbourne = ['South Melbourne']
st_kilda = ['St Kilda']
sydney = ['Sydney']
west_coast = ['West Coast']
western_bulldogs = ['Western Bulldogs', 'Footscray']
university = ['University']

In [13]:
def team_shorten(x):
    if x in adelaide:
        return "AD"
    elif x in brisbane:
        return "BR"
    elif x in carlton:
        return "CA"
    elif x in collingwood:
        return "CO"
    elif x in essendon:
        return "ES"
    elif x in fitzroy:
        return "FI"
    elif x in fremantle:
        return "FR"
    elif x in geelong:
        return "GE"
    elif x in gold_coast:
        return "GC"
    elif x in gws:
        return "GW"
    elif x in hawthorn:
        return "HA"
    elif x in melbourne:
        return "ME"
    elif x in north_melbourne:
        return "NM"
    elif x in port_adelaide:
        return "PA"
    elif x in richmond:
        return "RI"
    elif x in south_melbourne:
        return "SM"
    elif x in st_kilda:
        return "SK"
    elif x in sydney:
        return "SY"
    elif x in west_coast:
        return "WC"
    elif x in western_bulldogs:
        return "WB"
    elif x in university:
        return "UN"
    else: 
        return "NOT_FOUND"

In [14]:
trans["t1"] = trans["team1"].apply(team_shorten)
trans["t2"] = trans["team2"].apply(team_shorten)

In [15]:
trans['t1'].value_counts().sum()

15317

# 4. FIX TEAM NAMES ANOMOLIES

* Merge Footscray & Western Bulldogs as **Western Bulldogs**
* Merge North Melbourne with Kangaroos as **North Melbourne**
* Merge Brisbane Lions with Brisbane Bears as **Brisbane**
* Leave Sydney and **South Melbourne** as separate
* Leave **Fitzroy** as separate
* Modify GW Sydney to **GWS Giants**

In [16]:
# Function from Analytics Vidhya
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [17]:
trans["team1"] = coding(trans["team1"], {'Footscray':'Western Bulldogs'})
trans["team2"] = coding(trans["team2"], {'Footscray':'Western Bulldogs'})
trans["team1"] = coding(trans["team1"], {'Kangaroos':'North Melbourne'})
trans["team2"] = coding(trans["team2"], {'Kangaroos':'North Melbourne'})
trans["team1"] = coding(trans["team1"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team2"] = coding(trans["team2"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team1"] = coding(trans["team1"], {'GW Sydney':'GWS Giants'})
trans["team2"] = coding(trans["team2"], {'GW Sydney':'GWS Giants'})

In [18]:
trans["team2"].value_counts()

Collingwood         1268
Carlton             1247
Essendon            1218
Geelong             1213
Melbourne           1187
St Kilda            1186
Richmond            1115
North Melbourne      980
Hawthorn             972
Western Bulldogs     969
Fitzroy              960
South Melbourne      788
Sydney               426
West Coast           372
Brisbane             354
Adelaide             321
Fremantle            269
Port Adelaide        249
Gold Coast            84
GWS Giants            76
University            63
Name: team2, dtype: int64

# 5. ALL DRAWS (BOOLEAN)

In [19]:
def reg_draw(hvar):
    if hvar == 0:
        return 1
    else: return 0

In [20]:
trans["draw_rt"] = trans["hvar"].apply(reg_draw)

In [21]:
trans['draw_rt'].value_counts()

0    15159
1      158
Name: draw_rt, dtype: int64

### The ET draws

In [22]:
def et_draw(seq):
    if seq == 10794:
        return 1
    elif seq == 13203:
        return 1
    else: return 0

In [23]:
trans["etime_draw"] = trans["seq"].apply(et_draw)
trans['etime_draw'].value_counts()

0    15315
1        2
Name: etime_draw, dtype: int64

In [24]:
### Add the two columns
trans['draws_all']  = trans['draw_rt'] + trans['etime_draw']
trans['draws_all'].value_counts()

0    15157
1      160
Name: draws_all, dtype: int64

In [25]:
# Check
trans.iloc[:,:][(trans.etime_draw == 1)]

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,state,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all
10794,10794,1994-09-10,QF,North Melbourne,15,24,114,Hawthorn,13,13,...,VIC,23,205,1994,9,NM,HA,0,1,1
13203,13203,2007-09-14,SF,Collingwood,13,15,93,West Coast,10,14,...,WA,19,167,2007,9,CO,WC,0,1,1


# 6. GAME CATEGORY

In [26]:
finals = ['EF','QF','SF','PF','GF']

In [27]:
def game_cat(x):
    if x in finals:
        return "final"
    else: 
        return "reg_season"

In [28]:
trans["type"] = trans["round"].apply(game_cat)

In [29]:
trans['type'].value_counts()

reg_season    14671
final           646
Name: type, dtype: int64

In [30]:
trans.tail()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all,type
15313,15313,2018-06-22,R14,Port Adelaide,11,9,75,Melbourne,9,11,...,10,140,2018,6,PA,ME,0,0,0,reg_season
15314,15314,2018-06-23,R14,Hawthorn,13,18,96,Gold Coast,5,13,...,53,139,2018,6,HA,GC,0,0,0,reg_season
15315,15315,2018-06-23,R14,Brisbane,12,10,82,GWS Giants,16,13,...,-27,191,2018,6,BR,GW,0,0,0,reg_season
15316,15316,2018-06-23,R14,Western Bulldogs,11,9,75,North Melbourne,12,5,...,-2,152,2018,6,WB,NM,0,0,0,reg_season
15317,15317,2018-06-24,R14,Collingwood,11,13,79,Carlton,9,5,...,20,138,2018,6,CO,CA,0,0,0,reg_season


## 7. GROUPING COLUMNS

In [31]:
def grouping_2K(seq):
    n=500
    if seq <= 4*n:
        return 1
    elif seq <= 8*n:
        return 2
    elif seq <= 12*n:
        return 3
    elif seq <= 16*n:
        return 4
    elif seq <= 20*n:
        return 5
    elif seq <= 24*n:
        return 6
    elif seq <= 28*n:
        return 7
    else: return 8

In [32]:
trans["grp_2K"] = trans["seq"].apply(grouping_2K)

In [33]:
def grouping_1K(seq):
    n=1000
    if seq <= n:
        return 1
    elif seq <= 2*n:
        return 2
    elif seq <= 3*n:
        return 3
    elif seq <= 4*n:
        return 4
    elif seq <= 5*n:
        return 5
    elif seq <= 6*n:
        return 6
    elif seq <= 7*n:
        return 7
    elif seq <= 8*n:
        return 8
    elif seq <= 9*n:
        return 9
    elif seq <= 10*n:
        return 10
    elif seq <= 11*n:
        return 11
    elif seq <= 12*n:
        return 12
    elif seq <= 13*n:
        return 13
    elif seq <= 14*n:
        return 14
    elif seq <= 15*n:
        return 15
    else: return 16

In [34]:
trans["grp_1K"] = trans["seq"].apply(grouping_1K)

## 8. ADD res1 and res2 columns as categorical 1 = win, 2 = loss, 3 = draw based on hvar (ignoring the two extra time finals)

In [35]:
def result_cat_h(hvar):
    if hvar > 0:
        return 1
    elif hvar < 0:
        return 2
    else: return 3

In [36]:
def result_cat_a(hvar):
    if hvar > 0:
        return 2
    elif hvar < 0:
        return 1
    else: return 3

In [37]:
trans['res1'] = trans['hvar'].apply(result_cat_h)
trans['res2'] = trans['hvar'].apply(result_cat_a)

## 9. Reduce and re-order the data frame

In [38]:
tracker = trans
tracker = tracker[['year', 'team1','pts_1','team2','pts_2','venue','t1','t2','hvar']]
tracker.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar
15313,2018,Port Adelaide,75,Melbourne,65,Adelaide Oval,PA,ME,10
15314,2018,Hawthorn,96,Gold Coast,43,York Park,HA,GC,53
15315,2018,Brisbane,82,GWS Giants,109,Gabba,BR,GW,-27
15316,2018,Western Bulldogs,75,North Melbourne,77,Docklands,WB,NM,-2
15317,2018,Collingwood,79,Carlton,59,M.C.G.,CO,CA,20


### Create two new columns with values 1 and 0 for the two subsets

In [39]:
tracker['subset_1'] = 1
tracker['subset_2'] = 0

/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [40]:
tracker.head()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1,subset_2
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,0
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,0
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,0
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1,0
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1,0


### List for dropping 

In [41]:
droplist_1 = ['subset_2']
droplist_2 = ['subset_1']

In [42]:
left_team = tracker.drop(droplist_1, axis = 1)
right_team = tracker.drop(droplist_2, axis = 1)

In [43]:
left_team.shape

(15317, 10)

In [44]:
right_team.shape

(15317, 10)

### Rename columns in each dframe 

In [45]:
left_team.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1
15313,2018,Port Adelaide,75,Melbourne,65,Adelaide Oval,PA,ME,10,1
15314,2018,Hawthorn,96,Gold Coast,43,York Park,HA,GC,53,1
15315,2018,Brisbane,82,GWS Giants,109,Gabba,BR,GW,-27,1
15316,2018,Western Bulldogs,75,North Melbourne,77,Docklands,WB,NM,-2,1
15317,2018,Collingwood,79,Carlton,59,M.C.G.,CO,CA,20,1


In [46]:
left_team.columns = ['year','team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']

In [47]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15313,2018,Port Adelaide,75,Melbourne,65,Adelaide Oval,PA,ME,10,1
15314,2018,Hawthorn,96,Gold Coast,43,York Park,HA,GC,53,1
15315,2018,Brisbane,82,GWS Giants,109,Gabba,BR,GW,-27,1
15316,2018,Western Bulldogs,75,North Melbourne,77,Docklands,WB,NM,-2,1
15317,2018,Collingwood,79,Carlton,59,M.C.G.,CO,CA,20,1


In [48]:
right_team.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_2
15313,2018,Port Adelaide,75,Melbourne,65,Adelaide Oval,PA,ME,10,0
15314,2018,Hawthorn,96,Gold Coast,43,York Park,HA,GC,53,0
15315,2018,Brisbane,82,GWS Giants,109,Gabba,BR,GW,-27,0
15316,2018,Western Bulldogs,75,North Melbourne,77,Docklands,WB,NM,-2,0
15317,2018,Collingwood,79,Carlton,59,M.C.G.,CO,CA,20,0


In [49]:
right_team.columns = ['year','opp_team','op_pts','team','tm_pts','venue','op','tm','hvar','lr']

In [50]:
right_team.tail()

,year,opp_team,op_pts,team,tm_pts,venue,op,tm,hvar,lr
15313,2018,Port Adelaide,75,Melbourne,65,Adelaide Oval,PA,ME,10,0
15314,2018,Hawthorn,96,Gold Coast,43,York Park,HA,GC,53,0
15315,2018,Brisbane,82,GWS Giants,109,Gabba,BR,GW,-27,0
15316,2018,Western Bulldogs,75,North Melbourne,77,Docklands,WB,NM,-2,0
15317,2018,Collingwood,79,Carlton,59,M.C.G.,CO,CA,20,0


### Now re-order the second dframe to match columns

### Note double brackets to re-order

In [51]:
right_team = right_team[['year', 'team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']]
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15313,2018,Melbourne,65,Port Adelaide,75,Adelaide Oval,ME,PA,10,0
15314,2018,Gold Coast,43,Hawthorn,96,York Park,GC,HA,53,0
15315,2018,GWS Giants,109,Brisbane,82,Gabba,GW,BR,-27,0
15316,2018,North Melbourne,77,Western Bulldogs,75,Docklands,NM,WB,-2,0
15317,2018,Carlton,59,Collingwood,79,M.C.G.,CA,CO,20,0


In [52]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15313,2018,Port Adelaide,75,Melbourne,65,Adelaide Oval,PA,ME,10,1
15314,2018,Hawthorn,96,Gold Coast,43,York Park,HA,GC,53,1
15315,2018,Brisbane,82,GWS Giants,109,Gabba,BR,GW,-27,1
15316,2018,Western Bulldogs,75,North Melbourne,77,Docklands,WB,NM,-2,1
15317,2018,Collingwood,79,Carlton,59,M.C.G.,CO,CA,20,1


## CONCATENATION

### Now very easily, can concatenate the two dframes, as in si_week11

In [53]:
merge_ft = pd.concat([left_team, right_team], axis=0)
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1


In [54]:
merge_ft.shape

(30634, 10)

In [55]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15313,2018,Melbourne,65,Port Adelaide,75,Adelaide Oval,ME,PA,10,0
15314,2018,Gold Coast,43,Hawthorn,96,York Park,GC,HA,53,0
15315,2018,GWS Giants,109,Brisbane,82,Gabba,GW,BR,-27,0
15316,2018,North Melbourne,77,Western Bulldogs,75,Docklands,NM,WB,-2,0
15317,2018,Carlton,59,Collingwood,79,M.C.G.,CA,CO,20,0


In [56]:
merge_ft.sort_index(inplace=True)
merge_ft.head(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
3,1897,Essendon,47,Geelong,24,Corio Oval,ES,GE,-23,0
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
4,1897,Melbourne,44,South Melbourne,27,Lake Oval,ME,SM,-17,0
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1
5,1897,Carlton,36,South Melbourne,40,Lake Oval,CA,SM,4,0


## Create team result column

In [57]:
def neg_hvar(hvar):
        return -1*hvar

In [58]:
merge_ft["neg_dummy"] = merge_ft['hvar'].apply(neg_hvar)

In [59]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy
15315,2018,Brisbane,82,GWS Giants,109,Gabba,BR,GW,-27,1,27
15316,2018,North Melbourne,77,Western Bulldogs,75,Docklands,NM,WB,-2,0,2
15316,2018,Western Bulldogs,75,North Melbourne,77,Docklands,WB,NM,-2,1,2
15317,2018,Collingwood,79,Carlton,59,M.C.G.,CO,CA,20,1,-20
15317,2018,Carlton,59,Collingwood,79,M.C.G.,CA,CO,20,0,-20


In [60]:
def team_var(hvar):
    if merge_ft['lr'] == 1:
        return hvar
    else: return neg_dummy

In [61]:
def new_lr(lr):
    if lr == 0:
        return -1
    else: return 1

In [62]:
merge_ft["new_lr"] = merge_ft['lr'].apply(new_lr)
merge_ft["tm_var"] = merge_ft['new_lr']*merge_ft['hvar']

In [63]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy,new_lr,tm_var
15315,2018,Brisbane,82,GWS Giants,109,Gabba,BR,GW,-27,1,27,1,-27
15316,2018,North Melbourne,77,Western Bulldogs,75,Docklands,NM,WB,-2,0,2,-1,2
15316,2018,Western Bulldogs,75,North Melbourne,77,Docklands,WB,NM,-2,1,2,1,-2
15317,2018,Collingwood,79,Carlton,59,M.C.G.,CO,CA,20,1,-20,1,20
15317,2018,Carlton,59,Collingwood,79,M.C.G.,CA,CO,20,0,-20,-1,-20


### Can now remove the columns new_lr and neg_dummy, which were used to create tm_var

In [64]:
merge_ft = merge_ft.drop(['neg_dummy', 'new_lr'], axis=1)

In [65]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23
